# Tutorial 5: Sensitivity Analysis

## Understanding How Weights Affect Catalyst Rankings

This tutorial covers comprehensive sensitivity analysis for ASCI scoring:

1. Weight sweep analysis
2. Ternary diagrams showing optimal regions
3. Robustness metrics for catalyst selection
4. Identifying weight-insensitive top performers
5. Statistical correlation analysis

---

## 1. Setup and Data Loading

In [ ]:
import sys
sys.path.insert(0, '..')

from ascicat import ASCICalculator
from ascicat.sensitivity import SensitivityAnalyzer
from ascicat.config import REACTION_CONFIGS

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.tri as tri
from scipy import stats
import seaborn as sns
import os
from collections import defaultdict

plt.rcParams.update({
    'figure.dpi': 100,
    'font.size': 11,
    'axes.labelsize': 12,
    'axes.titlesize': 13
})

print("Setup complete!")

In [ ]:
# Load HER data
calc = ASCICalculator(reaction='HER', scoring_method='linear')
calc.load_data('../data/HER_clean.csv')

# Calculate baseline results (equal weights)
baseline_results = calc.calculate_asci(w_a=0.33, w_s=0.33, w_c=0.34)

print(f"Loaded {len(baseline_results)} catalysts")
print(f"Baseline ASCI (equal weights): mean={baseline_results['ASCI'].mean():.3f}")

## 2. Generate Weight Grid (Simplex Sampling)

Create a grid of weight combinations that sum to 1, covering the entire weight space.

In [ ]:
def generate_weight_grid(n_points=21, min_weight=0.1):
    """
    Generate weight combinations on the simplex.
    
    Parameters:
    -----------
    n_points : int, number of points along each dimension
    min_weight : float, minimum weight for each criterion
    
    Returns:
    --------
    list of (w_a, w_s, w_c) tuples
    """
    weights = []
    step = (1 - 3 * min_weight) / (n_points - 1) if n_points > 1 else 0
    
    for i in range(n_points):
        w_a = min_weight + i * step
        for j in range(n_points - i):
            w_s = min_weight + j * step
            w_c = 1 - w_a - w_s
            if w_c >= min_weight - 1e-9:  # Allow small numerical tolerance
                weights.append((round(w_a, 4), round(w_s, 4), round(w_c, 4)))
    
    return weights

# Generate weight grid
weight_grid = generate_weight_grid(n_points=21, min_weight=0.1)
print(f"Generated {len(weight_grid)} weight combinations")
print(f"\nSample weights:")
for w in weight_grid[:5]:
    print(f"  Activity={w[0]:.2f}, Stability={w[1]:.2f}, Cost={w[2]:.2f}")

## 3. Weight Sweep Analysis

Calculate ASCI scores for all weight combinations.

In [ ]:
# Run weight sweep
sweep_results = []

print("Running weight sweep...")
for idx, (w_a, w_s, w_c) in enumerate(weight_grid):
    # Create fresh calculator for each weight combination
    temp_calc = ASCICalculator(reaction='HER', scoring_method='linear')
    temp_calc.load_data('../data/HER_clean.csv')
    results = temp_calc.calculate_asci(w_a=w_a, w_s=w_s, w_c=w_c)
    
    # Get top catalyst
    top_1 = results.nsmallest(1, 'rank')
    
    sweep_results.append({
        'w_a': w_a,
        'w_s': w_s,
        'w_c': w_c,
        'top_catalyst': top_1['symbol'].values[0] if 'symbol' in top_1.columns else top_1.index[0],
        'top_ASCI': top_1['ASCI'].values[0],
        'mean_ASCI': results['ASCI'].mean(),
        'std_ASCI': results['ASCI'].std()
    })
    
    if (idx + 1) % 50 == 0:
        print(f"  Processed {idx + 1}/{len(weight_grid)} combinations...")

sweep_df = pd.DataFrame(sweep_results)
print(f"\nWeight sweep complete: {len(sweep_df)} combinations analyzed")

In [ ]:
# Analyze top catalyst frequency
catalyst_frequency = sweep_df['top_catalyst'].value_counts()

print("Top Catalyst Frequency Across Weight Space")
print("="*50)
print(f"{'Catalyst':<20} {'Frequency':<12} {'Percentage'}")
print("-"*50)
for cat, freq in catalyst_frequency.head(10).items():
    pct = 100 * freq / len(sweep_df)
    print(f"{cat:<20} {freq:<12} {pct:.1f}%")

## 4. Ternary Diagram: Weight Space Visualization

In [ ]:
def plot_ternary(w_a, w_s, w_c, values, title, cmap='viridis', label='Value'):
    """
    Create a ternary diagram for visualizing weight space.
    
    Parameters:
    -----------
    w_a, w_s, w_c : arrays of weights
    values : array of values to color by
    title : str, plot title
    cmap : str, colormap name
    label : str, colorbar label
    """
    fig, ax = plt.subplots(figsize=(10, 9))
    
    # Convert to cartesian coordinates for ternary plot
    # Using: x = w_s + 0.5*w_c, y = sqrt(3)/2 * w_c
    w_a = np.array(w_a)
    w_s = np.array(w_s)
    w_c = np.array(w_c)
    
    x = w_s + 0.5 * w_c
    y = np.sqrt(3) / 2 * w_c
    
    # Create triangulation
    triang = tri.Triangulation(x, y)
    
    # Plot contour
    tcf = ax.tricontourf(triang, values, levels=20, cmap=cmap)
    ax.tricontour(triang, values, levels=10, colors='white', linewidths=0.5, alpha=0.5)
    
    # Add scatter points
    ax.scatter(x, y, c=values, cmap=cmap, s=20, edgecolors='white', linewidths=0.5)
    
    # Draw triangle outline
    triangle = plt.Polygon([[0, 0], [1, 0], [0.5, np.sqrt(3)/2]], 
                           fill=False, edgecolor='black', linewidth=2)
    ax.add_patch(triangle)
    
    # Labels
    ax.text(0, -0.05, 'Activity', ha='center', va='top', fontsize=12, fontweight='bold')
    ax.text(1, -0.05, 'Stability', ha='center', va='top', fontsize=12, fontweight='bold')
    ax.text(0.5, np.sqrt(3)/2 + 0.05, 'Cost', ha='center', va='bottom', fontsize=12, fontweight='bold')
    
    # Colorbar
    cbar = plt.colorbar(tcf, ax=ax, shrink=0.7)
    cbar.set_label(label, fontsize=11)
    
    ax.set_xlim(-0.1, 1.1)
    ax.set_ylim(-0.15, 1.0)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
    
    return fig, ax

# Plot ternary diagram of top ASCI scores
fig, ax = plot_ternary(
    sweep_df['w_a'], 
    sweep_df['w_s'], 
    sweep_df['w_c'],
    sweep_df['top_ASCI'],
    'Best ASCI Score Across Weight Space',
    cmap='viridis',
    label='Top ASCI Score'
)

output_dir = '../results/tutorial_sensitivity'
os.makedirs(output_dir, exist_ok=True)
plt.savefig(f'{output_dir}/ternary_top_ASCI.png', dpi=300, bbox_inches='tight')
plt.show()

## 5. Robustness Analysis: Top-N Frequency

In [ ]:
# Calculate top-10 frequency for each catalyst
top_n = 10
top_n_counts = defaultdict(int)

for idx, (w_a, w_s, w_c) in enumerate(weight_grid):
    temp_calc = ASCICalculator(reaction='HER', scoring_method='linear')
    temp_calc.load_data('../data/HER_clean.csv')
    results = temp_calc.calculate_asci(w_a=w_a, w_s=w_s, w_c=w_c)
    
    top_catalysts = results.nsmallest(top_n, 'rank')
    
    if 'symbol' in top_catalysts.columns:
        for cat in top_catalysts['symbol']:
            top_n_counts[cat] += 1
    else:
        for cat in top_catalysts.index:
            top_n_counts[cat] += 1

# Convert to DataFrame
robustness_df = pd.DataFrame([
    {'catalyst': cat, 'top10_count': count, 'top10_frequency': count / len(weight_grid)}
    for cat, count in top_n_counts.items()
]).sort_values('top10_count', ascending=False)

print(f"Robustness Analysis: Top-10 Frequency")
print("="*60)
print(f"Catalysts appearing in top-10 across {len(weight_grid)} weight combinations:\n")
print(robustness_df.head(20).to_string(index=False))

In [ ]:
# Visualize robustness
fig, ax = plt.subplots(figsize=(12, 8))

top_20_robust = robustness_df.head(20)
colors = plt.cm.viridis(top_20_robust['top10_frequency'] / top_20_robust['top10_frequency'].max())

bars = ax.barh(range(len(top_20_robust)), top_20_robust['top10_frequency'], color=colors)
ax.set_yticks(range(len(top_20_robust)))
ax.set_yticklabels(top_20_robust['catalyst'])
ax.set_xlabel('Frequency in Top-10', fontsize=12)
ax.set_title('Most Robust Catalysts: Top-10 Frequency Across Weight Space', 
             fontsize=14, fontweight='bold')

# Add frequency labels
for i, (_, row) in enumerate(top_20_robust.iterrows()):
    ax.text(row['top10_frequency'] + 0.01, i, f"{row['top10_frequency']:.2f}", 
            va='center', fontsize=9)

ax.invert_yaxis()
ax.set_xlim(0, 1.1)

plt.tight_layout()
plt.savefig(f'{output_dir}/robustness_top10.png', dpi=300, bbox_inches='tight')
plt.show()

## 6. Rank Correlation Analysis

How consistent are rankings across different weight configurations?

In [ ]:
# Select representative weight configurations
representative_weights = [
    (0.33, 0.33, 0.34, 'Equal'),
    (0.6, 0.2, 0.2, 'Activity-focused'),
    (0.2, 0.6, 0.2, 'Stability-focused'),
    (0.2, 0.2, 0.6, 'Cost-focused'),
    (0.5, 0.3, 0.2, 'Activity-Stability'),
    (0.3, 0.2, 0.5, 'Activity-Cost')
]

# Calculate rankings for each configuration
rankings = {}

for w_a, w_s, w_c, name in representative_weights:
    temp_calc = ASCICalculator(reaction='HER', scoring_method='linear')
    temp_calc.load_data('../data/HER_clean.csv')
    results = temp_calc.calculate_asci(w_a=w_a, w_s=w_s, w_c=w_c)
    rankings[name] = results['rank'].values

# Calculate Kendall's Tau correlation matrix
config_names = [name for _, _, _, name in representative_weights]
n_configs = len(config_names)
tau_matrix = np.zeros((n_configs, n_configs))

for i, name1 in enumerate(config_names):
    for j, name2 in enumerate(config_names):
        tau, _ = stats.kendalltau(rankings[name1], rankings[name2])
        tau_matrix[i, j] = tau

# Plot heatmap
fig, ax = plt.subplots(figsize=(10, 8))

im = ax.imshow(tau_matrix, cmap='RdYlGn', vmin=0, vmax=1)

ax.set_xticks(range(n_configs))
ax.set_yticks(range(n_configs))
ax.set_xticklabels(config_names, rotation=45, ha='right')
ax.set_yticklabels(config_names)

# Add correlation values
for i in range(n_configs):
    for j in range(n_configs):
        text = ax.text(j, i, f"{tau_matrix[i, j]:.2f}",
                       ha='center', va='center', fontsize=10,
                       color='white' if tau_matrix[i, j] < 0.5 else 'black')

ax.set_title("Kendall's Tau Rank Correlation Between Weight Configurations",
             fontsize=14, fontweight='bold')

cbar = plt.colorbar(im, ax=ax, shrink=0.8)
cbar.set_label("Kendall's Tau", fontsize=11)

plt.tight_layout()
plt.savefig(f'{output_dir}/kendall_tau_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

## 7. ASCI Score Trajectories

Track how individual catalyst scores change across weight space.

In [ ]:
# Get top 10 catalysts from equal weights
top_10_baseline = baseline_results.nsmallest(10, 'rank')
if 'symbol' in top_10_baseline.columns:
    tracked_catalysts = top_10_baseline['symbol'].tolist()
else:
    tracked_catalysts = top_10_baseline.index.tolist()

# Create activity weight sweep (keeping stability=cost)
activity_weights = np.linspace(0.1, 0.8, 15)

trajectory_data = {cat: {'w_a': [], 'ASCI': [], 'rank': []} for cat in tracked_catalysts}

for w_a in activity_weights:
    w_s = (1 - w_a) / 2
    w_c = (1 - w_a) / 2
    
    temp_calc = ASCICalculator(reaction='HER', scoring_method='linear')
    temp_calc.load_data('../data/HER_clean.csv')
    results = temp_calc.calculate_asci(w_a=w_a, w_s=w_s, w_c=w_c)
    
    for cat in tracked_catalysts:
        if 'symbol' in results.columns:
            match = results[results['symbol'] == cat]
        else:
            match = results.loc[[cat]] if cat in results.index else pd.DataFrame()
        
        if not match.empty:
            trajectory_data[cat]['w_a'].append(w_a)
            trajectory_data[cat]['ASCI'].append(match['ASCI'].values[0])
            trajectory_data[cat]['rank'].append(match['rank'].values[0])

In [ ]:
# Plot ASCI trajectories
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

colors = plt.cm.tab10(np.linspace(0, 1, len(tracked_catalysts)))

# Panel A: ASCI Score trajectories
for i, cat in enumerate(tracked_catalysts):
    data = trajectory_data[cat]
    axes[0].plot(data['w_a'], data['ASCI'], 'o-', color=colors[i], 
                 linewidth=2, markersize=5, label=cat)

axes[0].set_xlabel('Activity Weight', fontsize=12)
axes[0].set_ylabel('ASCI Score', fontsize=12)
axes[0].set_title('A. ASCI Score vs Activity Weight', fontsize=13, fontweight='bold')
axes[0].legend(bbox_to_anchor=(1.02, 1), loc='upper left', fontsize=9)
axes[0].axvline(0.33, color='gray', linestyle='--', alpha=0.5, label='Equal weights')

# Panel B: Rank trajectories
for i, cat in enumerate(tracked_catalysts):
    data = trajectory_data[cat]
    axes[1].plot(data['w_a'], data['rank'], 'o-', color=colors[i], 
                 linewidth=2, markersize=5, label=cat)

axes[1].set_xlabel('Activity Weight', fontsize=12)
axes[1].set_ylabel('Rank', fontsize=12)
axes[1].set_title('B. Rank vs Activity Weight', fontsize=13, fontweight='bold')
axes[1].invert_yaxis()
axes[1].set_ylim(50, 0)
axes[1].axvline(0.33, color='gray', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.savefig(f'{output_dir}/score_trajectories.png', dpi=300, bbox_inches='tight')
plt.show()

## 8. Weight Region Analysis

Which catalyst is #1 in different regions of weight space?

In [ ]:
# Categorize weight space into regions
def categorize_weight(w_a, w_s, w_c):
    """Categorize weight combination into a named region."""
    dominant = max(w_a, w_s, w_c)
    if dominant == w_a and w_a > 0.4:
        return 'Activity-dominant'
    elif dominant == w_s and w_s > 0.4:
        return 'Stability-dominant'
    elif dominant == w_c and w_c > 0.4:
        return 'Cost-dominant'
    else:
        return 'Balanced'

sweep_df['region'] = sweep_df.apply(
    lambda row: categorize_weight(row['w_a'], row['w_s'], row['w_c']), axis=1
)

# Analyze top catalyst by region
print("Top Catalyst by Weight Region")
print("="*60)

for region in ['Activity-dominant', 'Stability-dominant', 'Cost-dominant', 'Balanced']:
    region_data = sweep_df[sweep_df['region'] == region]
    if len(region_data) > 0:
        top_in_region = region_data['top_catalyst'].value_counts().head(3)
        print(f"\n{region} ({len(region_data)} weight combinations):")
        for cat, count in top_in_region.items():
            pct = 100 * count / len(region_data)
            print(f"  {cat}: {count} times ({pct:.1f}%)")

In [ ]:
# Visualize region dominance
region_counts = sweep_df.groupby(['region', 'top_catalyst']).size().unstack(fill_value=0)

# Get top 5 catalysts overall
top_5_overall = catalyst_frequency.head(5).index.tolist()
region_counts_filtered = region_counts[top_5_overall]

fig, ax = plt.subplots(figsize=(12, 6))

region_counts_filtered.plot(kind='bar', ax=ax, width=0.8)
ax.set_xlabel('Weight Region', fontsize=12)
ax.set_ylabel('Count (as #1 catalyst)', fontsize=12)
ax.set_title('Top Catalyst Dominance by Weight Region', fontsize=14, fontweight='bold')
ax.legend(title='Catalyst', bbox_to_anchor=(1.02, 1), loc='upper left')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

plt.tight_layout()
plt.savefig(f'{output_dir}/region_dominance.png', dpi=300, bbox_inches='tight')
plt.show()

## 9. Export Sensitivity Results

In [ ]:
# Export all sensitivity analysis results

# Weight sweep results
sweep_df.to_csv(f'{output_dir}/weight_sweep_results.csv', index=False)

# Robustness ranking
robustness_df.to_csv(f'{output_dir}/robustness_ranking.csv', index=False)

# Kendall's Tau matrix
tau_df = pd.DataFrame(tau_matrix, index=config_names, columns=config_names)
tau_df.to_csv(f'{output_dir}/kendall_tau_matrix.csv')

print(f"Sensitivity analysis results saved to: {output_dir}")
print("\nFiles created:")
for f in sorted(os.listdir(output_dir)):
    print(f"  - {f}")

## 10. Summary and Recommendations

### Key Findings

In [ ]:
print("Sensitivity Analysis Summary")
print("="*60)

# Most robust catalyst
most_robust = robustness_df.iloc[0]
print(f"\n1. Most Robust Catalyst: {most_robust['catalyst']}")
print(f"   - In top-10 across {most_robust['top10_frequency']*100:.1f}% of weight combinations")

# Most frequent #1
most_frequent_1 = catalyst_frequency.head(1)
print(f"\n2. Most Frequent #1 Catalyst: {most_frequent_1.index[0]}")
print(f"   - Ranked #1 in {100*most_frequent_1.values[0]/len(sweep_df):.1f}% of configurations")

# Average rank correlation
avg_tau = tau_matrix[np.triu_indices(n_configs, k=1)].mean()
print(f"\n3. Average Rank Correlation (Kendall's Tau): {avg_tau:.3f}")
print(f"   - Rankings are {'highly' if avg_tau > 0.8 else 'moderately' if avg_tau > 0.6 else 'weakly'} consistent across configurations")

# Weight sensitivity
asci_range = sweep_df['top_ASCI'].max() - sweep_df['top_ASCI'].min()
print(f"\n4. Top ASCI Score Range: {sweep_df['top_ASCI'].min():.3f} - {sweep_df['top_ASCI'].max():.3f}")
print(f"   - Variation: {asci_range:.3f}")

### Recommendations for Weight Selection

Based on this sensitivity analysis:

1. **For robust results**: Choose catalysts that appear frequently in the top-10 across all weight configurations
2. **For application-specific**: Use domain knowledge to emphasize the most critical criterion
3. **For uncertainty**: Report rankings for multiple weight configurations (at minimum: equal, activity-focused, cost-focused)

---

**Congratulations!** You've completed the ASCICat tutorial series.